In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="0T6RFylOHeX4QjCixrSE")
project = rf.workspace("elijoel").project("lata-tqoqd")
version = project.version(1)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 138.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to LATA-1 in yolov11:: 100%|██████████| 2082/2082 [00:00<00:00, 9543.39it/s]


**Reasoning**:
The first step is to import the `yaml` library to load the `data.yaml` file and understand its structure, including class names and paths to the image datasets for training, validation, and testing.



In [10]:
model_filename = 'modelo_cnn_personalizado.h5'
model.save(model_filename)
print(f"El modelo ha sido guardado exitosamente en '{model_filename}'")

El modelo ha sido guardado exitosamente en 'modelo_cnn_personalizado.h5'


In [2]:
import yaml

# Path to the data.yaml file
data_yaml_path = '/content/LATA-1/data.yaml'

with open(data_yaml_path, 'r') as file:
    data_config = yaml.safe_load(file)

print("Data configuration loaded successfully:")
print(data_config)


Data configuration loaded successfully:
{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 3, 'names': ['danada', 'defectuosa', 'sana'], 'roboflow': {'workspace': 'elijoel', 'project': 'lata-tqoqd', 'version': 1, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/elijoel/lata-tqoqd/dataset/1'}}


In [3]:
import os

# Extract base directory from data_yaml_path to construct full paths
base_dir = os.path.dirname(data_yaml_path)

# Construct full paths for train, val, and test image directories
train_image_dir = os.path.join(base_dir, data_config['train']).replace('../', '') # Adjusting path for Roboflow structure
val_image_dir = os.path.join(base_dir, data_config['val']).replace('../', '')
test_image_dir = os.path.join(base_dir, data_config['test']).replace('../', '')

print(f"Training image directory: {train_image_dir}")
print(f"Validation image directory: {val_image_dir}")
print(f"Test image directory: {test_image_dir}")

# Initialize empty lists for images and labels
train_images = []
train_labels = []
val_images = []
val_labels = []
test_images = []
test_labels = []

print("Empty lists initialized for images and labels.")

Training image directory: /content/LATA-1/train/images
Validation image directory: /content/LATA-1/valid/images
Test image directory: /content/LATA-1/test/images
Empty lists initialized for images and labels.


In [4]:
import cv2
import numpy as np

# Define target image size
IMG_SIZE = (224, 224) # Standard size for many CNNs

def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Warning: Could not load image {image_path}")
        return None
    image = cv2.resize(image, IMG_SIZE)
    image = image / 255.0 # Normalize pixel values to [0, 1]
    return image

def get_label_from_yolo_txt(label_file_path):
    try:
        with open(label_file_path, 'r') as f:
            first_line = f.readline().strip()
            if first_line:
                # YOLO format: class_id center_x center_y width height
                class_id = int(first_line.split(' ')[0])
                return class_id
            else:
                print(f"Warning: Label file {label_file_path} is empty.")
                return None
    except FileNotFoundError:
        print(f"Warning: Label file {label_file_path} not found.")
        return None
    except ValueError:
        print(f"Warning: Invalid data in label file {label_file_path}.")
        return None

# Helper function to process a dataset split
def process_dataset_split(image_dir, data_images_list, data_labels_list):
    label_dir = image_dir.replace('images', 'labels')
    print(f"Processing images from {image_dir} and labels from {label_dir}")

    for img_name in os.listdir(image_dir):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(image_dir, img_name)
            label_name = os.path.splitext(img_name)[0] + '.txt'
            label_file_path = os.path.join(label_dir, label_name)

            processed_image = load_and_preprocess_image(image_path)
            label = get_label_from_yolo_txt(label_file_path)

            if processed_image is not None and label is not None:
                data_images_list.append(processed_image)
                data_labels_list.append(label)

# Process training data
process_dataset_split(train_image_dir, train_images, train_labels)
print(f"Loaded {len(train_images)} training images and labels.")

# Process validation data
process_dataset_split(val_image_dir, val_images, val_labels)
print(f"Loaded {len(val_images)} validation images and labels.")

# Process test data
process_dataset_split(test_image_dir, test_images, test_labels)
print(f"Loaded {len(test_images)} test images and labels.")

# Convert lists to NumPy arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)
val_images = np.array(val_images)
val_labels = np.array(val_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

print("Datasets converted to NumPy arrays:")
print(f"Train images shape: {train_images.shape}, Train labels shape: {train_labels.shape}")
print(f"Validation images shape: {val_images.shape}, Validation labels shape: {val_labels.shape}")
print(f"Test images shape: {test_images.shape}, Test labels shape: {test_labels.shape}")

Processing images from /content/LATA-1/train/images and labels from /content/LATA-1/train/labels
Loaded 725 training images and labels.
Processing images from /content/LATA-1/valid/images and labels from /content/LATA-1/valid/labels
Loaded 207 validation images and labels.
Processing images from /content/LATA-1/test/images and labels from /content/LATA-1/test/labels
Loaded 103 test images and labels.
Datasets converted to NumPy arrays:
Train images shape: (725, 224, 224, 3), Train labels shape: (725,)
Validation images shape: (207, 224, 224, 3), Validation labels shape: (207,)
Test images shape: (103, 224, 224, 3), Test labels shape: (103,)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the custom CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Add dropout for regularization
    Dense(data_config['nc'], activation='softmax') # Output layer with number of classes and softmax activation
])

# Print the model summary to review the architecture
model.summary()

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Define the custom CNN architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Add dropout for regularization
    Dense(data_config['nc'], activation='softmax') # Output layer with number of classes and softmax activation
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001), # Using Adam optimizer with a default learning rate
    loss='sparse_categorical_crossentropy', # Appropriate loss for integer labels and multi-class classification
    metrics=['accuracy'] # Metric to evaluate the model's performance
)

print("Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric.")

Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric.


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam

# Define the custom CNN architecture
model = Sequential([
    Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)), # Explicitly define Input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Add dropout for regularization
    Dense(data_config['nc'], activation='softmax') # Output layer with number of classes and softmax activation
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.001), # Using Adam optimizer with a default learning rate
    loss='sparse_categorical_crossentropy', # Appropriate loss for integer labels and multi-class classification
    metrics=['accuracy'] # Metric to evaluate the model's performance
)

print("Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric, and Input layer specified.")
model.summary()

Model compiled successfully with Adam optimizer, sparse_categorical_crossentropy loss, and accuracy metric, and Input layer specified.


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,347 (42.61 MB)

 Trainable params: 11,169,347 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
history = model.fit(
    train_images,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(val_images, val_labels)
)

print("Model training complete. Training history stored in 'history' variable.")

Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 15s 349ms/step - accuracy: 0.4608 - loss: 1.6774 - val_accuracy: 0.7585 - val_loss: 0.5936
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 71ms/step - accuracy: 0.8164 - loss: 0.5051 - val_accuracy: 0.8792 - val_loss: 0.3255
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 60ms/step - accuracy: 0.9014 - loss: 0.2470 - val_accuracy: 0.8696 - val_loss: 0.3115
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.9206 - loss: 0.2155 - val_accuracy: 0.9034 - val_loss: 0.1980
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 61ms/step - accuracy: 0.9629 - loss: 0.1019 - val_accuracy: 0.8986 - val_loss: 0.2663
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.9658 - loss: 0.1036 - val_accuracy: 0.9227 - val_loss: 0.2247
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.9682 - loss: 0.0762 - val_accuracy: 0.9324 - val_loss: 0.1353
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.9815 - loss: 0.0487 - val_accuracy: 0.9420 -

In [9]:
loss, accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 297ms/step - accuracy: 0.9322 - loss: 0.1764
Test Loss: 0.2375
Test Accuracy: 0.9320
